In [ ]:
%pip install crunch-cli --upgrade --quiet --progress-bar off
!crunch setup-notebook datacrunch J97ln71UCKG5f3hRedbKUFYT

In [ ]:
import os
import typing
# Import your dependencies
import joblib
import pandas as pd
import numpy as np
import scipy
import sklearn.metrics

In [ ]:
import crunch
# Load the Crunch Toolings
crunch = crunch.load_notebook()
# Run a local test
crunch.test()

In [ ]:
# Load the data simply
# X_train, y_train, X_test = crunch.load_data()
from preprocess import  load_data
X_train, X_test, y_train, y_test = load_data()

In [ ]:
list_stats_0, list_stats_1 = [], []
wasserstein_list, ks_list = [], []

q_vals = np.linspace(0.001, 0.999, 30)

for id in range(len(y_train)):
    if id in [1000, 2_000, 5_000, 9_000]:
        print(f'X_id {id}')
    X_id = X_train.loc[id]
    stat_0_0 = X_id[X_id['period']==0]['value'].quantile(q_vals).values
    stat_0_1 = X_id[X_id['period']==1]['value'].quantile(q_vals).values
    list_stats_0.append(stat_0_0)
    list_stats_1.append(stat_0_1)
    wasserstein_list.append(np.mean(abs(stat_0_0 - stat_0_1)))
    ks_list.append(np.max(abs(stat_0_0 - stat_0_1)))


In [ ]:
import matplotlib.pyplot as plt

ks =np.array(ks_list)
wass = np.array(wasserstein_list)
plt.scatter(ks,wass, alpha=0.5)
plt.scatter(ks[y_train.values==1],wass[y_train.values==1], alpha=0.9)
plt.show()

In [12]:
from extract_features import *
Nf= 5
Z0_ff = f_x2z_fourier(X_train[X_train['period']==0],Nf)
Z1_ff = f_x2z_fourier(X_train[X_train['period']==1],Nf)
Z0_ff['it_breaks'] = y_train
Z1_ff['it_breaks'] = y_train

In [13]:
# Z0_mom = f_x2z_moments(X_train[X_train['period']==0])
# Z1_mom = f_x2z_moments(X_train[X_train['period']==1])
# Z0_mom['it_breaks'] = y_train
# Z1_mom['it_breaks'] = y_train

from preprocess import X_to_wide
# build wide train + align target
X_train_wide = X_to_wide(X_train)
X_train_wide = X_train_wide.loc[y_train.index]  # ensure same ids


In [14]:
# build wide test + align target test
X_test_wide = X_to_wide(X_test)
X_test_wide = X_test_wide.loc[y_test.index]  # ensure same ids


In [15]:
X_test_wide

time,p_t0,p_t1,p_t10,p_t100,p_t1000,p_t1001,p_t1002,p_t1003,p_t1004,p_t1005,...,v_t990,v_t991,v_t992,v_t993,v_t994,v_t995,v_t996,v_t997,v_t998,v_t999
id,,,,,,,,,,,,,,,,,,,,,
10001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.054945,-0.010417,0.042105,0.010101,-0.010000,-0.010101,-0.061224,0.010870,-0.021505,0.010989
10002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.021645,-0.023305,-0.002169,-0.036957,0.038375,0.017391,0.034188,-0.022727,0.044397,0.109312
10003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.054054,0.064102,-0.012048,-0.036585,0.012658,-0.012500,-0.012658,0.012820,0.012658,-0.025000
10004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012739,0.018239,-0.000618,-0.012979,0.038823,-0.034358,-0.014357,0.016466,0.003115,-0.001242
10005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.001894,-0.000661,0.006840,0.001934,-0.005219,-0.000047,-0.003799,-0.000417,-0.012627,0.002923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10097,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.012615,0.001877,0.005108,-0.011506,-0.008653,-0.008936,-0.001612,-0.000257,0.009739,-0.000043
10098,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.003664,-0.013538,0.006897,-0.021783,-0.011121,0.020953,0.031353,0.013536,-0.004509,-0.011185
10099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.026715,0.014400,0.013269,-0.002756,0.002596,-0.027875,-0.013182,-0.000908,0.026989,-0.028461


In [ ]:
Z_pca_train , Z_pcs_test = fit_pca_and_transform(X_train,X_test)

In [ ]:
Z_pca_train , Z_pcs_test = fit_pca_and_transform(X_train,X_test)

In [ ]:
plt.ecdf(Z0_ff[Z0_ff['it_breaks'] ==1]['fpow_2'])
plt.ecdf(Z1_ff[Z1_ff['it_breaks'] ==1]['fpow_2'])

In [ ]:
import seaborn as sns
sns.pairplot(Z1_ff, hue='it_breaks')

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split


# fill NaNs with 0 (or could use mean imputation)
X_train_wide = X_train_wide.fillna(0.0)

# quick train/valid split (if you don't want to use X_test yet)
Xtr, Xval, ytr, yval = train_test_split(
    X_train_wide, y_train, test_size=0.2, stratify=y_train, random_state=42
)

clf = LogisticRegression(
    max_iter=10000,
    solver="saga",          # handles L1/L2 on large sparse-ish data
    penalty="l2",
    C=1.0,
    class_weight="balanced",
    n_jobs=-1
)

clf.fit(Xtr, ytr)

# validation ROC-AUC
p_val = clf.predict_proba(Xval)[:, 1]
print("Validation ROC-AUC:", roc_auc_score(yval, p_val))

# ===== apply to test set =====
X_test_wide = X_to_wide(X_test).fillna(0.0)
X_test_wide = X_test_wide.loc[y_test.index]  # align
p_test = clf.predict_proba(X_test_wide)[:, 1]

print("Test ROC-AUC:", roc_auc_score(y_test, p_test))
